## notebooks/03_diagnostic_outputs.ipynb - DIAGNOSTIC AVANCÉ
# Outil de debug et validation pour pipelines P11
# ============================================================================
# 1 - CONFIGURATION ET IMPORTS
# ============================================================================

In [1]:
import os
import sys
import glob
import warnings
from pathlib import Path
from pprint import pprint
import pandas as pd

warnings.filterwarnings('ignore')
sys.path.append('../src')

# Import modules P11
from emr_simulation import get_spark_session, EMRSimulation, get_s3_client

print("🔍 DIAGNOSTIC AVANCÉ DES OUTPUTS P11")
print("="*45)

🔍 DIAGNOSTIC AVANCÉ DES OUTPUTS P11


# ============================================================================
# 2 - SÉLECTION DU MODE À DIAGNOSTIQUER
# ============================================================================

## 2.1 - Configuration du mode à diagnostiquer

In [2]:
MODE_TO_DIAGNOSE = "local"  # "local", "simulation", "production"

print(f"🎯 Mode de diagnostic : {MODE_TO_DIAGNOSE}")

🎯 Mode de diagnostic : local


## 2.2 - Initialisation session Spark adaptée

In [3]:
spark = get_spark_session(mode=MODE_TO_DIAGNOSE, app_name="DiagnosticOutputs")
emr_config = EMRSimulation(mode=MODE_TO_DIAGNOSE)

2025-07-31 07:45:45.065383: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-31 07:45:46.667413: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2025-07-31 07:45:48.502850: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-07-31 07:45:49.179542: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-07-31 07:45:49.179600: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_exec

🚀 GPU détecté automatiquement
🔧 EMRSimulation initialisé en mode: local
🚀 Optimisations GPU activées


25/07/31 07:45:51 WARN Utils: Your hostname, PC-ARNAUD resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/07/31 07:45:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/nono66/P11/2-python/venv_p11/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/nono66/.ivy2/cache
The jars for the packages stored in: /home/nono66/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-de6ec85a-fdeb-4248-b040-b684f5493625;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.696 in central
:: resolution report :: resolve 191ms :: artifacts dl 6ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.696 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 by [com.amazonaws#aws-java-sdk-bundle;1.12.696] in [default]
	----------------------------------------------------------------

✅ Session Spark créée - Mode: local - Version: 3.4.1
📊 Config mémoire: 6g executor / 4g driver
🎯 Cores disponibles: 16
🔧 EMRSimulation initialisé en mode: local


# ============================================================================
# 3 - DIAGNOSTIC RÉPERTOIRES LOCAUX
# ============================================================================

In [4]:
def diagnose_local_outputs():
    """Diagnostic complet du répertoire outputs/ local"""
    print("\n📁 DIAGNOSTIC RÉPERTOIRE LOCAL")
    print("-" * 35)
    
    outputs_dir = Path("../outputs")
    
    if not outputs_dir.exists():
        print("❌ Répertoire outputs/ introuvable")
        return
    
    # 1. Structure globale
    total_files = list(outputs_dir.rglob("*"))
    total_size_mb = sum([f.stat().st_size for f in total_files if f.is_file()]) / (1024*1024)
    
    print(f"📊 Statistiques globales :")
    print(f"   - Fichiers totaux : {len([f for f in total_files if f.is_file()])}")
    print(f"   - Dossiers : {len([f for f in total_files if f.is_dir()])}")
    print(f"   - Taille totale : {total_size_mb:.1f} MB")
    
    # 2. Fichiers Parquet
    parquet_files = list(outputs_dir.rglob("*.parquet"))
    print(f"\n📦 FICHIERS PARQUET ({len(parquet_files)}) :")
    
    parquet_info = {}
    for pf in parquet_files:
        try:
            # Lecture avec Spark pour valider
            df = spark.read.parquet(str(pf))
            count = df.count()
            schema_cols = len(df.columns)
            size_mb = pf.stat().st_size / (1024*1024) if pf.is_file() else 0
            
            # Si c'est un dossier parquet, calculer taille totale
            if pf.is_dir():
                size_mb = sum([f.stat().st_size for f in pf.rglob("*") if f.is_file()]) / (1024*1024)
            
            parquet_info[str(pf.relative_to(outputs_dir))] = {
                'rows': count,
                'columns': schema_cols,
                'size_mb': size_mb,
                'schema': df.columns
            }
            
            print(f"   ✅ {pf.name}")
            print(f"      📊 {count:,} lignes, {schema_cols} colonnes, {size_mb:.1f} MB")
            
        except Exception as e:
            print(f"   ❌ {pf.name} - Erreur : {str(e)[:50]}...")
    
    # 3. Fragments Spark (part-*.parquet)
    part_files = list(outputs_dir.rglob("part-*.parquet"))
    crc_files = list(outputs_dir.rglob("*.crc"))
    
    print(f"\n🔧 FRAGMENTS SPARK :")
    print(f"   - Fichiers part-* : {len(part_files)}")
    print(f"   - Fichiers .crc : {len(crc_files)}")
    
    if part_files:
        print("   📂 Répertoires avec partitions :")
        partition_dirs = set([f.parent for f in part_files])
        for pd in partition_dirs:
            part_count = len(list(pd.glob("part-*.parquet")))
            print(f"      - {pd.relative_to(outputs_dir)} : {part_count} partitions")
    
    # 4. Graphiques et images
    image_files = []
    for ext in ['*.png', '*.jpg', '*.svg', '*.pdf']:
        image_files.extend(outputs_dir.rglob(ext))
    
    if image_files:
        print(f"\n🖼️ GRAPHIQUES ET IMAGES ({len(image_files)}) :")
        for img in image_files:
            size_mb = img.stat().st_size / (1024*1024)
            print(f"   - {img.relative_to(outputs_dir)} ({size_mb:.2f} MB)")
    
    return parquet_info

# ============================================================================
# 4 - DIAGNOSTIC S3 (SIMULATION/PRODUCTION)
# ============================================================================

In [5]:
def diagnose_s3_outputs():
    """Diagnostic complet du stockage S3"""
    print(f"\n☁️ DIAGNOSTIC S3 - MODE {MODE_TO_DIAGNOSE.upper()}")
    print("-" * 40)
    
    try:
        s3_client = get_s3_client(mode=MODE_TO_DIAGNOSE)
        bucket_name = "fruits-p11-simulation" if MODE_TO_DIAGNOSE == "simulation" else "fruits-p11-production"
        
        # Liste des objets
        response = s3_client.list_objects_v2(Bucket=bucket_name)
        
        if 'Contents' not in response:
            print("📦 Bucket vide ou inaccessible")
            return
        
        objects = response['Contents']
        total_size_mb = sum([obj['Size'] for obj in objects]) / (1024*1024)
        
        print(f"📊 Statistiques S3 :")
        print(f"   - Bucket : {bucket_name}")
        print(f"   - Objets totaux : {len(objects)}")
        print(f"   - Taille totale : {total_size_mb:.1f} MB")
        
        # Classification par type
        object_types = {
            'results': [],
            'cache': [],
            'plots': [],
            'test': [],
            'other': []
        }
        
        for obj in objects:
            key = obj['Key']
            size_mb = obj['Size'] / (1024*1024)
            
            if key.startswith('results/') or 'final_results' in key:
                object_types['results'].append((key, size_mb))
            elif key.startswith('cache/'):
                object_types['cache'].append((key, size_mb))
            elif key.startswith('plots/'):
                object_types['plots'].append((key, size_mb))
            elif key.startswith('test/'):
                object_types['test'].append((key, size_mb))
            else:
                object_types['other'].append((key, size_mb))
        
        # Affichage organisé
        for obj_type, obj_list in object_types.items():
            if obj_list:
                print(f"\n📁 {obj_type.upper()} ({len(obj_list)}) :")
                for key, size in sorted(obj_list, key=lambda x: x[1], reverse=True):
                    print(f"   - {key} ({size:.2f} MB)")
        
        # Test de lecture d'un fichier Parquet S3
        parquet_keys = [obj['Key'] for obj in objects if obj['Key'].endswith('.parquet') and 'final_results' in obj['Key']]
        
        if parquet_keys:
            print(f"\n🔍 TEST LECTURE PARQUET S3 :")
            test_key = parquet_keys[0]
            try:
                s3_path = f"s3a://{bucket_name}/{test_key}"
                df_test = spark.read.parquet(s3_path)
                count = df_test.count()
                cols = len(df_test.columns)
                print(f"   ✅ {test_key}")
                print(f"      📊 {count:,} lignes, {cols} colonnes")
                print(f"      🔗 Path : {s3_path}")
            except Exception as e:
                print(f"   ❌ Erreur lecture {test_key} : {str(e)[:60]}...")
    
    except Exception as e:
        print(f"❌ Erreur connexion S3 : {e}")
        print("💡 Vérifiez LocalStack (simulation) ou credentials AWS (production)")

# ============================================================================
# 5 - VALIDATION INTÉGRITÉ DONNÉES
# ============================================================================

In [6]:
def validate_pipeline_integrity():
    """Validation de l'intégrité complète du pipeline"""
    print(f"\n🔍 VALIDATION INTÉGRITÉ PIPELINE")
    print("-" * 35)
    
    # Fichiers critiques attendus
    critical_files = {
        'local': [
            '../outputs/final_results.parquet',
            '../outputs/cache/pca_variance_analysis.parquet',
            '../outputs/cache/features_mobilenet_gpu.parquet'
        ],
        'simulation': [
            'final_results.parquet',
            'cache/pca_variance_analysis_optimized.parquet',
            'results/features_pca_optimal.parquet'
        ],
        'production': [
            'final_results.parquet', 
            'cache/pca_variance_analysis_optimized.parquet',
            'results/features_pca_optimal.parquet'
        ]
    }
    
    expected_files = critical_files.get(MODE_TO_DIAGNOSE, [])
    
    print(f"🎯 Fichiers critiques attendus ({len(expected_files)}) :")
    
    validation_results = {}
    
    for file_path in expected_files:
        exists = False
        valid = False
        row_count = 0
        
        try:
            if MODE_TO_DIAGNOSE == 'local':
                # Vérification locale
                exists = os.path.exists(file_path) or os.path.exists(file_path.replace('.parquet', ''))
                if exists:
                    df = spark.read.parquet(file_path)
                    row_count = df.count()
                    valid = row_count > 0
            else:
                # Vérification S3
                bucket_name = "fruits-p11-simulation" if MODE_TO_DIAGNOSE == "simulation" else "fruits-p11-production"
                s3_path = f"s3a://{bucket_name}/{file_path}"
                
                try:
                    df = spark.read.parquet(s3_path)
                    row_count = df.count()
                    exists = True
                    valid = row_count > 0
                except:
                    exists = False
                    valid = False
            
            status = "✅" if (exists and valid) else "❌"
            print(f"   {status} {file_path}")
            if exists and valid:
                print(f"      📊 {row_count:,} lignes")
            
            validation_results[file_path] = {
                'exists': exists,
                'valid': valid,
                'rows': row_count
            }
            
        except Exception as e:
            print(f"   ❌ {file_path} - Erreur : {str(e)[:50]}...")
            validation_results[file_path] = {'exists': False, 'valid': False, 'rows': 0}
    
    # Résumé validation
    total_files = len(expected_files)
    valid_files = sum([1 for v in validation_results.values() if v['valid']])
    
    print(f"\n📊 RÉSUMÉ VALIDATION :")
    print(f"   - Fichiers attendus : {total_files}")
    print(f"   - Fichiers valides : {valid_files}")
    print(f"   - Statut pipeline : {'✅ OPÉRATIONNEL' if valid_files == total_files else '⚠️ INCOMPLET'}")
    
    return validation_results

# ============================================================================
# 6 - EXÉCUTION DU DIAGNOSTIC
# ============================================================================
## 6.1 - Exécution selon le mode

In [7]:
if MODE_TO_DIAGNOSE == "local":
    parquet_info = diagnose_local_outputs()
else:
    diagnose_s3_outputs()


📁 DIAGNOSTIC RÉPERTOIRE LOCAL
-----------------------------------
📊 Statistiques globales :
   - Fichiers totaux : 246
   - Dossiers : 14
   - Taille totale : 12.2 MB

📦 FICHIERS PARQUET (119) :


   ✅ final_results.parquet
      📊 500 lignes, 3 colonnes, 0.3 MB
   ✅ features_pca_optimal.parquet
      📊 500 lignes, 4 colonnes, 4.0 MB
   ✅ pca_variance_analysis.parquet
      📊 56 lignes, 3 colonnes, 0.0 MB
   ✅ pca_variance_analysis_gpu.parquet
      📊 35 lignes, 3 colonnes, 0.0 MB
   ✅ features_mobilenet_gpu.parquet
      📊 500 lignes, 3 colonnes, 3.7 MB
   ✅ images_paths.parquet
      📊 500 lignes, 2 colonnes, 0.0 MB
   ✅ features_mobilenet.parquet
      📊 500 lignes, 3 colonnes, 3.7 MB
   ✅ part-00015-acc09db0-a2ba-4009-b029-19615234baa1-c000.snappy.parquet
      📊 5 lignes, 3 colonnes, 0.0 MB
   ✅ part-00014-acc09db0-a2ba-4009-b029-19615234baa1-c000.snappy.parquet
      📊 3 lignes, 3 colonnes, 0.0 MB
   ✅ part-00013-acc09db0-a2ba-4009-b029-19615234baa1-c000.snappy.parquet
      📊 3 lignes, 3 colonnes, 0.0 MB
   ✅ part-00004-acc09db0-a2ba-4009-b029-19615234baa1-c000.snappy.parquet
      📊 3 lignes, 3 colonnes, 0.0 MB
   ✅ part-00010-acc09db0-a2ba-4009-b029-19615234baa1-c000.sn

## 6.2 - Validation dans tous les cas

In [8]:
validation_results = validate_pipeline_integrity()


🔍 VALIDATION INTÉGRITÉ PIPELINE
-----------------------------------
🎯 Fichiers critiques attendus (3) :
   ✅ ../outputs/final_results.parquet
      📊 500 lignes
   ✅ ../outputs/cache/pca_variance_analysis.parquet
      📊 56 lignes
   ✅ ../outputs/cache/features_mobilenet_gpu.parquet
      📊 500 lignes

📊 RÉSUMÉ VALIDATION :
   - Fichiers attendus : 3
   - Fichiers valides : 3
   - Statut pipeline : ✅ OPÉRATIONNEL


# ============================================================================
# 7 - RÉSUMÉ ET RECOMMANDATIONS
# ============================================================================

In [9]:
print(f"\n🎓 RÉSUMÉ DIAGNOSTIC - MODE {MODE_TO_DIAGNOSE.upper()}")
print("="*50)


🎓 RÉSUMÉ DIAGNOSTIC - MODE LOCAL


## 7.1 - Calcul des métriques globales

In [10]:
total_files_found = 0
total_valid_files = sum([1 for v in validation_results.values() if v['valid']])
total_expected = len(validation_results)

if MODE_TO_DIAGNOSE == "local":
    local_outputs = Path("../outputs")
    if local_outputs.exists():
        total_files_found = len([f for f in local_outputs.rglob("*") if f.is_file()])

print(f"📊 Métriques :")
print(f"   - Mode diagnostiqué : {MODE_TO_DIAGNOSE}")
print(f"   - Fichiers critiques trouvés : {total_valid_files}/{total_expected}")
print(f"   - Statut global : {'✅ SAIN' if total_valid_files == total_expected else '⚠️ À VÉRIFIER'}")

if total_files_found > 0:
    print(f"   - Total fichiers détectés : {total_files_found}")

print(f"\n💡 RECOMMANDATIONS :")
if total_valid_files == total_expected:
    print("   ✅ Pipeline en bon état, prêt pour la suite")
    if MODE_TO_DIAGNOSE == "local":
        print("   🚀 Vous pouvez tester les modes simulation/production")
    elif MODE_TO_DIAGNOSE == "simulation":
        print("   ☁️ Prêt pour migration AWS EMR production")
else:
    missing_files = [k for k, v in validation_results.items() if not v['valid']]
    print(f"   ⚠️ Fichiers manquants/corrompus : {', '.join(missing_files[:3])}")
    print("   🔄 Relancez le notebook correspondant pour régénérer")

print(f"\n🔧 Session Spark active - Mode: {MODE_TO_DIAGNOSE}")
print("💡 Arrêtez avec : spark.stop()")

📊 Métriques :
   - Mode diagnostiqué : local
   - Fichiers critiques trouvés : 3/3
   - Statut global : ✅ SAIN
   - Total fichiers détectés : 246

💡 RECOMMANDATIONS :
   ✅ Pipeline en bon état, prêt pour la suite
   🚀 Vous pouvez tester les modes simulation/production

🔧 Session Spark active - Mode: local
💡 Arrêtez avec : spark.stop()


# ============================================================================
# 8 - FONCTIONS UTILITAIRES BONUS
# ============================================================================

In [11]:
def quick_parquet_inspect(file_path):
    """Inspection rapide d'un fichier Parquet spécifique"""
    try:
        if MODE_TO_DIAGNOSE != "local":
            bucket_name = "fruits-p11-simulation" if MODE_TO_DIAGNOSE == "simulation" else "fruits-p11-production"
            file_path = f"s3a://{bucket_name}/{file_path}"
        
        df = spark.read.parquet(file_path)
        print(f"\n🔍 INSPECTION : {file_path}")
        print(f"📊 Lignes : {df.count():,}")
        print(f"📊 Colonnes : {len(df.columns)}")
        print("📊 Schéma :")
        df.printSchema()
        print("📊 Aperçu :")
        df.show(5)
        
    except Exception as e:
        print(f"❌ Erreur inspection {file_path} : {e}")

# Exemple d'utilisation :
# quick_parquet_inspect("final_results.parquet")

In [12]:
print("\n✅ Diagnostic terminé. Utiliser quick_parquet_inspect('fichier.parquet') pour inspection détaillée.")


✅ Diagnostic terminé. Utiliser quick_parquet_inspect('fichier.parquet') pour inspection détaillée.
